In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
train_angry=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_angry.npy")
train_disgust=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_disgust.npy")
train_fear=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_fear.npy")
train_happy=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_happy.npy")
train_neutral=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_neutral.npy")
train_sad=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_sad.npy")
train_surprise=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/train_img_surprise.npy")

In [3]:
train_angry.shape, train_disgust.shape, train_fear.shape, train_happy.shape, train_neutral.shape, train_sad.shape, train_surprise.shape

((3993, 48, 48, 3),
 (436, 48, 48, 3),
 (4103, 48, 48, 3),
 (7164, 48, 48, 3),
 (4982, 48, 48, 3),
 (4938, 48, 48, 3),
 (3205, 48, 48, 3))

In [4]:
val_angry=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_angry.npy")
val_disgust=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_disgust.npy")
val_fear=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_fear.npy")
val_happy=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_happy.npy")
val_neutral=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_neutral.npy")
val_sad=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_sad.npy")
val_surprise=np.load("/home/lab/양창희/Emotion Project/emotion_image_npy/validation_img_surprise.npy")

In [5]:
angry=np.concatenate((train_angry,val_angry),axis=0)
disgust=np.concatenate((train_disgust,val_disgust),axis=0)
fear=np.concatenate((train_fear,val_fear),axis=0)
happy=np.concatenate((train_happy,val_happy),axis=0)
neutral=np.concatenate((train_neutral,val_neutral),axis=0)
sad=np.concatenate((train_sad,val_sad),axis=0)
surprise=np.concatenate((train_surprise,val_surprise),axis=0)

In [6]:
angry.shape

(4953, 48, 48, 3)

In [7]:
label=[]
for i in range(7):
    if i==0:
        for j in range(len(angry)):
            label.append(0)
    elif i==1:
        for j in range(len(disgust)):
            label.append(1)
    elif i==2:
        for j in range(len(fear)):
            label.append(2)
    elif i==3:
        for j in range(len(happy)):
            label.append(3)
    elif i==4:
        for j in range(len(neutral)):
            label.append(4)
    elif i==5:
        for j in range(len(sad)):
            label.append(5)
    elif i==6:
        for j in range(len(surprise)):
            label.append(6)

In [8]:
input_data=np.concatenate((angry,disgust,fear,happy,neutral,sad,surprise),axis=0)

In [9]:
label=np.array(label)
label.shape, input_data.shape

((35887,), (35887, 48, 48, 3))

In [10]:
input_data= (input_data*(1/256)-0.5)/0.5

In [11]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [13]:
label.shape

torch.Size([35887])

In [49]:
input_data=input_data.transpose(1,3)

In [50]:
input_data.shape

torch.Size([35887, 3, 48, 48])

In [51]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [52]:
trainset,valset=torch.utils.data.random_split(dataset,[30887,5000])
trainset,testset=torch.utils.data.random_split(trainset,[25887,5000])

partition={'train':trainset,'val':valset,'test':testset}

In [97]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=3,
                            out_channels=32,
                            kernel_size=6,
                            stride=2,
                            padding=2) # 32 24 24
        self.conv2=nn.Conv2d(in_channels=32, 
                            out_channels=64,
                            kernel_size=5,
                            stride=1,
                            padding=2) # 64 60 64
        self.conv3=nn.Conv2d(in_channels=64,
                            out_channels=128,
                            kernel_size=3,
                            stride=1,
                            padding=1) # 25 30 128
        self.act=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size=2,
                                  stride=2)
        self.fc=nn.Linear(1152,7)
    def forward(self,x):
        x=self.conv1(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=self.conv2(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=self.conv3(x)
        x=self.act(x)
        x=self.maxpool1(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        return x

In [98]:
def dimension_check():
    net=CNN()
    x=torch.randn(1,3,48,48)
    y=net(x)
    print(y.size())

In [99]:
dimension_check()

torch.Size([1, 7])


In [100]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=64,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [101]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=64, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [102]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=64, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [103]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [105]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=CNN()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  7%|▋         | 27/405 [00:00<00:01, 268.59it/s]

time : 1.85, train_loss : 1.61, train_acc : 36.32,val_acc : 44.74,val_loss : 1.46


  7%|▋         | 29/405 [00:00<00:01, 285.81it/s]

time : 1.58, train_loss : 1.37, train_acc : 47.49,val_acc : 48.30,val_loss : 1.34


  7%|▋         | 29/405 [00:00<00:01, 283.63it/s]

...

  7%|▋         | 27/405 [00:00<00:01, 264.37it/s]

time : 1.57, train_loss : 0.13, train_acc : 96.85,val_acc : 50.54,val_loss : 7.26


  0%|          | 0/79 [00:00<?, ?it/s]

time : 1.56, train_loss : 0.16, train_acc : 96.39,val_acc : 51.48,val_loss : 7.66


100%|██████████| 79/79 [00:00<00:00, 370.10it/s]


In [107]:
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Basic_val_acc.npy",val_acc)